In [ ]:
##########################################
####        Importing Pachages        ####
##########################################

from __future__ import division, print_function
import kf_book.pf_internal as pf_internal
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform
from filterpy.stats import plot_gaussian_pdf
import csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
from collections import Counter
import pickle
import math as m
import sys
import scipy
from scipy.spatial import distance
from numpy.random import uniform
from operator import itemgetter
from filterpy.monte_carlo import systematic_resample
from numpy.linalg import norm
from numpy.random import randn
import scipy.stats
import time
import paho.mqtt.client as mqtt
import threading
import os
import re

gridsize = 1
EPSILON = sys.float_info.epsilon  # smallest possible difference

##########################################
####      MQTT Broker Definitions     ####
##########################################

    




##########################################
####  PF Internal Methods Definitions ####
##########################################

def simple_resample(particles, weights):
    N = len(particles)
    cumulative_sum = np.cumsum(weights)
    cumulative_sum[-1] = 1. # avoid round-off error
    indexes = np.searchsorted(cumulative_sum, random(N))

    # resample according to indexes
    particles[:] = particles[indexes]
    weights.fill(1.0 / N)
    
def resample_from_index(particles, weights, indexes):
    particles[:] = particles[indexes]
    weights[:] = weights[indexes]
    weights.fill(1.0 / len(weights))    

def neff(weights):
    return 1. / np.sum(np.square(weights))

def create_uniform_particles(x_range, y_range, hdg_range, N):
    particles = np.empty((N, 3))
    particles[:, 0] = uniform(x_range[0], x_range[1], size=N)
    particles[:, 1] = uniform(y_range[0], y_range[1], size=N)
    particles[:, 2] = uniform(hdg_range[0], hdg_range[1], size=N)
    particles[:, 2] %= 2 * np.pi
    return particles

def create_gaussian_particles(mean, std, N):
    particles = np.empty((N, 3))
    particles[:, 0] = mean[0] + (randn(N) * std[0])
    particles[:, 1] = mean[1] + (randn(N) * std[1])
    particles[:, 2] = mean[2] + (randn(N) * std[2])
    particles[:, 2] %= 2 * np.pi
    return particles


def DistanceToRSSI(agent_pos):
    rssi_values = []
    for b in beacons:
        dist = int(round(np.linalg.norm(b - agent_pos)))
        
        # Max reading is (7 / gridsize) meters
        if (dist > int(7 / gridsize)):
            dist = int(7 / gridsize)
        
        # print(dist)
        mean = beacons_mean[dist - 1]
        std = beacons_stds[dist - 1]
        
        sample = np.random.normal(mean, std, 1)
        rssi_values.append(sample[0])
        
    return rssi_values

#------------------------------------
#----------------------- Update Step:

def SensorModel(distances):
    m = itemgetter(*distances)(beacons_mean)
    s = itemgetter(*distances)(beacons_stds)
    
    return [m, s]

def update(particles, weights, z, R):
    for i, beacon in enumerate(beacons):
        distances = np.round((np.linalg.norm(particles[:, 0:2] - beacon, axis=1))/10).astype(int)
        ms, ss = SensorModel(distances)
        
        # weighting function
        weights += np.abs(np.log10(scipy.stats.norm(ms, ss).pdf(z[i])))
        
    weights += 1.e-300      # avoid round-off to zero
    
#------------------------------------
#------------- State Estimation Step:  
    
def estimate(particles, weights):
    """returns mean and variance of the weighted particles"""
    pos = particles[:, 0:2]
    mean = np.average(pos, weights=weights, axis=0)
    var  = np.average((pos - mean)**2, weights=weights, axis=0)
    return mean, var    

#------------------------------------
#------------- Particle Predict Step:  

def predict(particles, xbins, ybins):
    next_states = NextStates(TM, particles, xbins, ybins)
    particles[:, 0] = next_states[:, 0]
    particles[:, 1] = next_states[:, 1]
    return particles

#------------------------------------
#----------------- Transition Matrix: 

def transition_matrix(transitions, xbins, ybins):
    M = np.ones((xbins * ybins, xbins * ybins))
    for (state, state_p) in zip(transitions,transitions[1:]):
        i = xbins * state[1] + state[0]
        j = xbins * state_p[1] + state_p[0]
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

def CumulativeSum(m):
    return m.cumsum()

def returnobj(cdf):
    randnum = np.random.random()
    for i in range(len(cdf) - 1):
        if((randnum>cdf[i]) and (randnum<=cdf[i + 1])):
            return i

def IndexToState(index, xbins, ybins):
    y = int(np.floor(index / xbins))
    x = index - xbins * y
    return x, y

def StateToIndex(x, y, xbins, ybins):
    i = xbins * np.floor(y) + np.floor(x)
    return int(i)
        
def NextStates(TM, particles, xbins, ybins):
    cdf = np.apply_along_axis(CumulativeSum, 1, TM)
    zero = np.array([0]) # np.zeros((len(cdf[:,1]))
    c = np.tile(zero[np.newaxis,:], (cdf.shape[0],1))
    cdf = np.concatenate((c, cdf), axis=1)
    next_states = np.empty((len(particles), 2))
    
    for i in range(len(particles)):
        current_state_x = particles[i, 0]
        current_state_y = particles[i, 1]
        
        row = StateToIndex(current_state_x, current_state_y, xbins, ybins)
        next_state_index = returnobj(cdf[row])
        next_states[i, 0] = IndexToState(next_state_index, xbins, ybins)[0]
        next_states[i, 1] = IndexToState(next_state_index, xbins, ybins)[1]

    return next_states

def beacon_sensor_model():
    filename = '../RSSI.csv'

agenttrace = 'AgentTrace.csv'
f = open(agenttrace)
oracle_raw = []
gridized_data = []

# Information Granularity
realx = 6.6
realy = 10.6                
xbins = int(np.ceil(realx) / gridsize)
ybins = int(np.ceil(realy) / gridsize)

with open(agenttrace, 'r') as user_data:
    reader = csv.reader(user_data, delimiter=',', quotechar=',')
    for row in reader:
        if (reader.line_num > 1):
            oracle_raw.append([float(row[2]), float(row[3]), str(row[5])])

oracle = []
for i in range(len(oracle_raw)):
     oracle.append([oracle_raw[i][0] / gridsize, oracle_raw[i][1] / gridsize , oracle_raw[i][2]])
            
for d in oracle:
    gridized_data.append([int(np.floor(d[0])), int(np.floor(d[1]))])        

beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])    

for i in range(len(beacons)):
    beacons[i][0] = beacons[i][0]/gridsize
    beacons[i][1] = beacons[i][1]/gridsize

TM = transition_matrix(gridized_data, xbins, ybins)    

##########################################
####     Sensors class Definitions    ####
##########################################

class PressureSensor:
    radius = float()
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, r, obj):
        self.x = x
        self.y = y
        self.radius = r
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False
        
class WaterFlowSensor:
    x = float()
    y = float()
    attached_object = str()
    active = bool()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj

    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class VoltageSensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True

    def Deactive(self):
        self.active = False
        
class ElectricitySensor:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.active = False
        self.attached_object = obj
    
    def Fire(self):
        self.active = True
    
    def Deactive(self):
        self.active = False
    
class Accelerometer:
    x = float()
    y = float()
    active = bool()
    attached_object = str()
    
    def Initialize(self, x, y, obj):
        self.x = x
        self.y = y
        self.attached_object = obj
        self.active = False
        
    def Fire(self):
        self.active = True
        
    def Deactive(self):
        self.active = False
        
##########################################
####  Space and Sensor Model Config.  ####
##########################################

def interpolate(inp, fi):
    i = int(fi)
    f = fi - i
    return (inp[i] if f < EPSILON else
            inp[i] + f*(inp[i+1]-inp[i]))


beacons = np.array([[1, 1], [3, 4.9], [1.35, 7.7], [3.3, 10.5], [3.3, 10.5], [6, 0.5], [0.5, 3.3], [6, 10.5]])
activity_sensors = []
NL = len(beacons)

pik = 'SensorModel.dat'

with open(pik, 'rb') as f:
    beacons_mean = pickle.load(f)
    beacons_stds = pickle.load(f)
    
new_len = int(len(beacons_mean) / gridsize)
delta = (len(beacons_mean)-1) / float(new_len-1)
new_beacons_mean = [interpolate(beacons_mean, i*delta) for i in range(new_len)]

new_len = int(len(beacons_stds) / gridsize)
delta = (len(beacons_stds)-1) / float(new_len-1)
new_beacons_stds = [interpolate(beacons_stds, i*delta) for i in range(new_len)]

beacons_mean = new_beacons_mean
beacons_stds = new_beacons_stds

sensordefs = 'BIMSensorDef.csv'
f = open(sensordefs)
oracle_raw = []
gridized_data = []

with open(sensordefs, 'r') as sensor:
    reader = csv.reader(sensor, delimiter=',', quotechar=',')
    for row in reader:
        # print(row)
        if(row[0] == 'pressure'):
            pressure_sensor = PressureSensor()
            pressure_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, 2 / gridsize, row[1])
            activity_sensors.append(pressure_sensor)
        
        elif(row[0] == 'water'):
            water_sensor = WaterFlowSensor()
            water_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(water_sensor)
        
        elif(row[0] == 'electricity'):
            voltage_sensor = VoltageSensor()
            voltage_sensor.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(voltage_sensor)
        
        elif(row[0] == 'accelerometer'):
            acc = Accelerometer()
            acc.Initialize(float(row[2]) / gridsize, float(row[3]) / gridsize, row[1])
            activity_sensors.append(acc)

##########################################
####   Activity Recognition Methods   ####
##########################################            

def ActivitiesSensorGeneration(data):
    for sensor in activity_sensors:
        activity = data[2].lower()
        if (activity == 'sleep' or activity == 'sit' or activity == 'eat' or activity == 'entertainment'):
            if(type(sensor) == PressureSensor):
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2):
                    sensor.Fire()

        elif (activity == 'wash' or activity == 'clean'):
            if(type(sensor) == WaterFlowSensor):
                if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 2):
                    sensor.Fire()

        elif (activity == 'exercise'):
            if(type(sensor) == Accelerometer):
                if(sensor.attached_object == 'Agent'):
                    sensor.Fire()
                    
        elif (activity == 'cook'):
            if(type(sensor) == Accelerometer):
                if(sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    if(distance.euclidean([float(sensor.x), float(sensor.y)], [data[0], data[1]]) <= 1 / gridsize):
                        sensor.Fire()
        
        elif (activity == 'cook'):
            if (type(sensor) == ElectricitySensor):
                if (sensor.attached_object == 'Oven Tower Unit' or sensor.attached_object == 'Counter Top w Sink'
                   or sensor.attached_object == 'Counter Top - Corner'):
                    sensor.Fire()
                
        elif (activity == 'entertainment'):
            if(type(sensor) == ElectricitySensor):
                if(sensor.attached_object == 'TV - Flat Screen'):
                    sensor.Fire()

def ResetSensorStates():
    for sensor in activity_sensors:
        sensor.Deactive()
        
def RecognizeActivity(current_location, previous_location):
    for sensor in activity_sensors:
        if (sensor.active):
            if (type(sensor) == Accelerometer):
                if (sensor.attached_object == 'Refrigerator-Side-by-Side-with_Ice_and_Water_Dispenser'):
                    return 'cook'
                
                elif (sensor.attached_object == 'Agent'):
                    return 'exercise'
            
            elif (type(sensor) == ElectricitySensor):
                if (sensor.attached_object == 'Oven Tower Unit' or sensor.attached_object == 'Counter Top w Sink' 
                    or sensor.attached_object == 'Counter Top - Corner'):
                    return 'cook'
                
                if (sensor.attached_object == 'TV - Flat Screen'):
                    return 'entertainment'
                
            elif (type(sensor) == WaterFlowSensor):
                if (sensor.attached_object == 'Sink Kitchen-Single'):
                    return 'clean'
                
            elif (type(sensor) == PressureSensor):
                if (sensor.attached_object == 'Chair-Breuer' or sensor.attached_object == 'Dining Table w Chairs'):
                    return 'sit'
                
                elif (sensor.attached_object == 'Chair-Corbu' or sensor.attached_object == 'Sofa-Corbu'
                      or sensor.attached_object == 'bed'):
                    return 'sleep'
                
    if (distance.euclidean(current_location, previous_location) <= 0.05):
        return 'nothing'
        
    else:
        return 'walk'
            
    
##########################################
####     Particle Filter Main Body    ####
##########################################      

def run_pf1(N, iters=18, sensor_std_err=.1, do_plot=True, plot_particles=False, 
            xlim=(0, xbins), ylim=(0, ybins), initial_x=None):
    fig, ax = plt.subplots(1,1)
    particles = create_uniform_particles((0,xbins), (0,ybins), (0, 1.28), N)    
    weights = np.ones(N) / N
    xs = []
    step = 1
    for gt in range(len(oracle)):
        step = step + 1
        agent_trace = oracle[gt]
        agent_pos = [agent_trace[0], agent_trace[1]]
        
        ActivitiesSensorGeneration(agent_trace)
        
        particles = predict(particles, xbins, ybins)        
        particles[0][0] = agent_pos[0]
        particles[0][1] = agent_pos[1]
        
        zs =  DistanceToRSSI(agent_pos)
        update(particles, weights, z = zs, R = sensor_std_err)
        
        # resample if too few effective particles
        if neff(weights) < N/2:
            indexes = systematic_resample(weights)
            resample_from_index(particles, weights, indexes)
            assert np.allclose(weights, 1/N)
        mu, var = estimate(particles, weights)
        xs.append(mu)
        
        p1 = ax.scatter(agent_pos[0], agent_pos[1], marker='+', color='k', s=180, lw=3)
        p2 = ax.scatter(mu[0], mu[1], marker='s', color='r')
    
        
        if plot_particles:
            alpha = .2
            if N > 5000:
                alpha *= np.sqrt(5000)/np.sqrt(N)           
            plt.scatter(particles[:, 0], particles[:, 1], alpha=alpha, color='b')
        
        xs_new = np.array(xs)
        plt.legend([p1, p2], ['Synthetic Ground Truth', 'Particle Filter Estimation'], loc=1, numpoints=1)
        plt.xlim(*xlim)
        plt.ylim(*ylim)
        fig.suptitle("Synthetic Ground Truth Activity: " + 
                     str(agent_trace[2]) + 
                     " |  Estimated Activity: " + 
                     str(RecognizeActivity(xs[len(xs) - 1], xs[len(xs) - 2])))

        fig.canvas.draw()
        time.sleep(0.3)
        ax.cla()
        ResetSensorStates()
        
%matplotlib notebook

from numpy.random import seed
seed(2) 
run_pf1(N=100, plot_particles=False)

In [ ]:
import paho.mqtt.client as mqtt
import threading
import os
import re


def on_connect(client, userdata, flags, rc):
    client.subscribe("/ADLSimulator/#")

def on_message(client, userdata, msg):
    result = msg.topic+"  :  " + str(msg.payload)
    print(result)

    # pass the result to localization

def on_publish(client, userdata, result):
    # print("Data published :" + str(client) + " " + str(userdata) + str(result) + "\n")
    # ParseResult(result)
    pass

def on_subscribed(mosq, obj, mid, granted_qos):
    print("Subscribed mid: " + str(mid) + ", qos: " + str(granted_qos))
    
    
port = 1883
host = "localhost"
keepalive = 60
broker_topic = "#"  #All topics have to be read



client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.on_publish = on_publish
client.on_subscribed = on_subscribed

client.connect(host, port, 60)
client.subscribe(broker_topic, 0)

client.loop_start()